In [ ]:
#default_exp plotting

In [ ]:
#exporti
import warnings
import numpy as np
import torch
import plotly.graph_objs as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import pyvista as pv
from typing import Union

from dl4to.utils import cast_to_problem

pv.set_jupyter_backend('pythreejs')

In [ ]:
#hide
from nbdev.showdoc import show_doc

# 3D plotting

In [ ]:
#exporti
class Voxels(go.Mesh3d):
    """
    A class that generates 3d voxelized mesh objects.
    """
    def __init__(self, 
                 problem:"dl4to.problem.Problem", # The problem that is referred to for plotting.
                 scalar_field:torch.Tensor, # The scalar field that determines the presence or absence of voxels.
                 data:torch.Tensor, # The data that should be represented in color on the voxels.
                 colorscale:str=None, # The name of the `plotly` colorscale that is used for plotting.
                 threshold:float=.5, # The threshold below which voxels are not displayed, i.e., represented as void.
                 **kwargs):
        width=(
            problem.size[0] / (problem.shape[0] - 1),
            problem.size[1] / (problem.shape[1] - 1),
            problem.size[2] / (problem.shape[2] - 1),
        )

        positions, scalar_field, data = self._filter_and_reshape_voxel_information(problem, scalar_field, data, threshold)

        # Create a cuboid at the origin as the basic construction unit
        x_base = np.array([0, 0, 1, 1, 0, 0, 1, 1]) * width[0]
        y_base = np.array([0, 1, 1, 0, 0, 1, 1, 0]) * width[1]
        z_base = np.array([0, 0, 0, 0, 1, 1, 1, 1]) * width[2]
        i_base = np.array([7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2])
        j_base = np.array([3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3])
        k_base = np.array([0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6])

        x_s, y_s, z_s = np.empty(0), np.empty(0), np.empty(0)
        i_s, j_s, k_s = np.empty(0), np.empty(0), np.empty(0)
        intensity = np.empty(0)

        # Create a cuboid for each voxel
        for idx, (position, scalar_field_, data_) in enumerate(zip(positions, scalar_field, data)):
            x_s = np.append(x_s, x_base + position[0])
            y_s = np.append(y_s, y_base + position[1])
            z_s = np.append(z_s, z_base + position[2])
            i_s = np.append(i_s, i_base + 8 * idx)
            j_s = np.append(j_s, j_base + 8 * idx)
            k_s = np.append(k_s, k_base + 8 * idx)
            intensity = np.append(intensity, np.ones(8) * data_)

        super().__init__(x=x_s, y=y_s, z=z_s, i=i_s, j=j_s, k=k_s, intensity=intensity, colorscale=colorscale, **kwargs)


    def _filter_and_reshape_voxel_information(self, problem, scalar_field, data, threshold=.5):
        X, Y, Z = np.mgrid[
            0 : problem.size[0] : complex(problem.shape[0]),
            0 : problem.size[1] : complex(problem.shape[1]),
            0 : problem.size[2] : complex(problem.shape[2]),
        ]

        positions = np.array([X.flatten(), Y.flatten(), Z.flatten()]).T

        positions_new = []
        scalar_field_new = []
        data_new = []

        for position, scalar_field_, data_ in zip(positions, scalar_field.flatten(), data.flatten()):
            if scalar_field_ > threshold:
                positions_new.append(position)
                scalar_field_new.append(scalar_field_)
                data_new.append(data_)

        positions = np.array(positions_new)
        scalar_field = np.array(scalar_field_new)
        data = np.array(data_new)

        return positions, scalar_field, data

In [ ]:
#export
def plot_scalar_field(
    problem:"dl4to.problem.Problem", # The problem that is referred to for plotting.
    scalar_field:torch.Tensor, # The scalar field that determines the presence or absence of voxels.
    data:torch.Tensor=None, # The data that should be represented in color on the voxels.
    title:str=None, # The name of the plot.
    file_path:str=None, # Path where the figure is saved.
    threshold:float=.5, # The threshold below which voxels are not displayed, i.e., represented as void.
    display:bool=True, # Whether the figure is displayed.
    camera_position:Union[tuple,list]=(0,.1,.12), # x, y, and z coordinates of the camera position.
    show_design_space:bool=False, # Whether to highlight the voxels that have a design space information of -1 assigned to them.
    show_colorbar:bool=True, # Determines whether a reference colorbar is displayed for the plotted voxel color values.
    colorscale:str=None, # The name of the colorscale that is used for plotting.
    show_axislabels:bool=False, # Whether the 3d axes are labelled with their dimensions.
    show_ticklabels:bool=False, # Whether the 3d axes ticks are displayed and labeled.
    export_png:bool=False, # Whether the figure is exported and saved as a png file, in addition to the standard html format.
    **kwargs
):
    """
    Plots a three-dimensional scalar field visualization. Returns a `plotly.graph_objs.FigureWidget` object.
    """

    shape = problem.shape
    scalar_field = scalar_field.reshape(*shape)

    if data is None:
        data = scalar_field
    else:
        data = data.reshape(*shape)

    voxels = Voxels(problem, scalar_field, data, colorscale=colorscale, threshold=threshold,
                    **kwargs)
    if show_design_space:
        design_space = problem.Ω_design.reshape(*shape) == -1
        voxels_design_space = Voxels(problem, design_space, design_space, colorscale=None,
                                     threshold=threshold, opacity=0.3, **kwargs)
    layout = go.Layout(title=title)
    if show_design_space:
        fig = go.FigureWidget(data=[voxels, voxels_design_space], layout=layout)
    else:
        fig = go.FigureWidget(data=[voxels], layout=layout)

    ranges = np.vstack([[0, 0, 0], problem.size]).T
    ranges[:,1] = (ranges[:,1] - ranges[:,0]) * np.array(shape) / (np.array(shape) - 1)

    camera = dict(
        eye=dict(x=camera_position[0], y=camera_position[1], z=camera_position[2])
    )

    if show_axislabels:
        axislabels = ['x', 'y', 'z']
    else:
        axislabels = ['', '', '']

    fig.update_layout(
        scene_camera=camera,
        scene = dict(xaxis = dict(range=ranges[0], title=axislabels[0], showticklabels=show_ticklabels),
                     yaxis = dict(range=ranges[1], title=axislabels[1], showticklabels=show_ticklabels),
                     zaxis = dict(range=ranges[2], title=axislabels[2], showticklabels=show_ticklabels)),
        scene_aspectmode='manual',
        scene_aspectratio=dict(x=ranges[0][1] - ranges[0][0],
                               y=ranges[1][1] - ranges[1][0],
                               z=ranges[2][1] - ranges[2][0]),
    )
    fig.update_traces(showscale=show_colorbar)

    if display:
        fig.show()

    if file_path is not None:
        if export_png:
            fig.write_image(f"{file_path}.png")
        plot(fig, filename=f"{file_path}.html", auto_open=False)
    if file_path is None and export_png:
        warnings.warn("Can only export png if a file_path is provided.")
    return fig

In [ ]:
#export
def pyvista_plot_scalar_field(
    problem:"dl4to.problem.Problem", # The problem that is referred to for plotting.
    scalar_field:torch.Tensor, # The scalar field that determines the presence or absence of voxels.
    data:torch.Tensor=None, # The data that should be represented in color on the voxels.
    title:str=None, # The name of the plot.
    threshold:float=.5, # The threshold below which voxels are not displayed, i.e., represented as void.
    display:bool=True, # Whether the figure is displayed.
    camera_position:Union[tuple,list]=(0,.1,.12), # x, y, and z coordinates of the camera position.
    show_design_space:bool=False, # Whether to highlight the voxels that have a design space information of -1 assigned to them.
    window_size:Union[tuple,list]=(800,800), # The size of the window that displays the plot
    smooth_iters:int=0, # The number of smoothing iterations for better looking visualizations. Note that the number of smoothing iterations can be reduced if Taubin smoothing is used. This is because it is more efficient than Laplacian smoothing.
    taubin_smoothing:bool=True, # Whether to use Taubin smoothing. If `False`, then Laplacian smoothing is used. Taubin smoothing has the advantage that it is volume preserving, i.e., a higher number of smoothing iterations does not lead to thiner structures. Only has an effect if `smooth_iters > 0`.
    taubin_pass_band:float=.1, # The passband value for the windowed sinc filter. This should be between 0 and 2, where lower values cause more smoothing.
    show_edges:bool=True, # Whether to display the edges of the underlying grid.
):
    """
    Plots a three-dimensional scalar field visualization using pyvista. Returns a `pyvista.plotting.plotting.Plotter` object.
    Pyvista plotting leads to better looking visualizations than the standard plotly interface. This is mainly due to the fact, that pyvista integrates Taubin smoothing [1].
    However, the pyvista interface also has its downsides: In our experiments we found that only the backend 'pythreejs' worked for us. 
    This backend unfortunately currently does not support several basic functionalities. This includes for instance the display of color bars and plot titles as well as the option to save generated plots. 
    Therefore, the plots need to be saved via manual screenshots. We still decided to leave this class a part of the DL4TO library, since the visualizations
    are much better looking for publications than the default plotly interface. it is possible that the missing features will be added in the future, if pyvista adds them to their
    pythreejs backend or we somehow manage to make a different backend work.
    """
    scalar_field = np.where(scalar_field < threshold+1e-9, 0., 1.)
    if len(scalar_field.shape) == 4:
        scalar_field = scalar_field.squeeze(0)
    grid = pv.UniformGrid()
    grid.dimensions = np.array(scalar_field.shape) + 1
    grid.spacing = problem.h
    grid.cell_data["scalar_field"] = scalar_field.flatten(order="F")
    if data is not None:
        
        grid.cell_data["data"] = data.flatten(order="F")
    grid.cell_data["design"] = problem.Ω_design.squeeze(0).detach().cpu().numpy().flatten(order="F") != 0
    grid.cell_data["back"] = np.ones_like(problem.Ω_design.squeeze(0).detach().cpu().numpy().flatten(order="F"))

    scalar_field = grid.threshold(threshold+1e-9, scalars='scalar_field')
    design = grid.threshold(0.5, scalars='design')
    back = grid.threshold(0.5, scalars='back')

    if smooth_iters > 0:
        surf = scalar_field.extract_surface()
        surf_design = design.extract_surface()
        if taubin_smoothing:
            scalar_field = surf.smooth_taubin(n_iter=smooth_iters, pass_band=taubin_pass_band)
            design = surf_design.smooth_taubin(n_iter=smooth_iters, pass_band=taubin_pass_band)
        else:
            scalar_field = surf.smooth(n_iter=smooth_iters)
            design = surf_design.smooth_taubin(n_iter=smooth_iters, pass_band=taubin_pass_band)
        if show_edges:
            edges = surf.extract_feature_edges()
    elif show_edges:
        edges = scalar_field.extract_feature_edges()

    p = pv.Plotter(title=title)
    p.set_background('white')

    if show_design_space:
        p.add_mesh(design, scalars='design', cmap='Greys', show_edges=False, opacity=0.3)
    else:
        p.add_mesh(back, scalars='back', cmap='Greys', show_edges=False, opacity=0.0, render=False)
    if data is not None:
        p.add_mesh(scalar_field, scalars='data', cmap='cool', show_edges=show_edges, show_scalar_bar=True)
    else:
        p.add_mesh(scalar_field, scalars='scalar_field', cmap='Greys', show_edges=show_edges)
    if display:
        p.show(cpos=camera_position, window_size=window_size)
    return p

In [ ]:
#export
def plot_vector_field(
    problem:"dl4to.problem.Problem", # The problem that is referred to for plotting.
    vector_field:torch.Tensor, # The vector field that should be displayed with 3d arrows.
    title:str=None, # The name of the plot.
    file_path:str=None, # Path where the figure is saved.
    display:bool=True, # Whether the figure is displayed.
    camera_position:Union[tuple,list]=(0,.1,.12), # x, y, and z coordinates of the camera position.
    show_colorbar:bool=True, # Determines whether a reference colorbar is displayed for the plotted voxel color values.
    colorscale:str='Blues', # The name of the colorscale that is used for plotting.
    show_axislabels:bool=False, # Whether the 3d axes are labelled with their dimensions.
    show_ticklabels:bool=False, # Whether the 3d axes ticks are displayed and labeled.
    export_png:bool=False # Whether the figure is exported and saved as a png file, in addition to the standard html format.
):
    """
    Plots a three-dimensional vector field visualization. Returns a `plotly.graph_objs.FigureWidget` object.
    """
    indices = np.nonzero(vector_field)[:,-3:].numpy()
    indices = np.vstack([indices, [0, 0, 0]])
    indices = np.vstack([indices, [problem.shape[0]-1, problem.shape[1]-1, problem.shape[2]-1]])

    X = indices[:,0]
    Y = indices[:,1]
    Z = indices[:,2]
    U = vector_field[0, X, Y, Z].numpy()
    V = vector_field[1, X, Y, Z].numpy()
    W = vector_field[2, X, Y, Z].numpy()

    layout = go.Layout(title=title)

    fig = go.FigureWidget(data = go.Cone(
        x=X, y=Y, z=Z, u=U, v=V, w=W,
        colorscale=colorscale, sizemode='absolute'
    ), layout=layout)

    camera = dict(
        eye=dict(x=camera_position[0], y=camera_position[1], z=camera_position[2])
    )

    if show_axislabels:
        axislabels = ['x', 'y', 'z']
    else:
        axislabels = ['', '', '']

    fig.update_layout(scene_camera=camera,
                      scene=dict(xaxis = dict(title=axislabels[0], showticklabels=show_ticklabels),
                                 yaxis = dict(title=axislabels[1], showticklabels=show_ticklabels),
                                 zaxis = dict(title=axislabels[2], showticklabels=show_ticklabels),
                                 aspectratio=dict(x=problem.size[0],
                                                  y=problem.size[1],
                                                  z=problem.size[2])))

    fig.update_traces(showscale=show_colorbar)

    if display:
        fig.show()

    if file_path is not None:
        if export_png:
            fig.write_image(f"{file_path}.png")
        plot(fig, filename=f"{file_path}.html", auto_open=False)
    if file_path is None and export_png:
        warnings.warn("Can only export png if a file_path is provided.")

    return fig

# References

[1] Taubin, Gabriel. "Curve and surface smoothing without shrinkage." Proceedings of IEEE international conference on computer vision. IEEE, 1995.

In [ ]:
%%time
#hide

from dl4to.datasets import BasicDataset
import torch

def test_plot(verbose):
    problem = BasicDataset().ledge()
    scalar_field = torch.rand(problem.shape)
    data = torch.rand(problem.shape)

    if verbose:
        fig = plot_scalar_field(problem, scalar_field, data, display=True)


test_plot(verbose=False)

CPU times: user 627 ms, sys: 147 ms, total: 773 ms
Wall time: 1.35 s
